In [1]:
import gc
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

from tqdm import tqdm
from datetime import datetime, timedelta, time

In [2]:
# os.makedirs('./temp', exist_ok=True)

# TEMP_PATH = './temp/'

In [3]:
%%time

train = pd.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/train.parquet')
train['ts'] = pd.to_datetime(train['ts']*1e9)

cutoff_w1 = train.ts.min() + timedelta(days = 6, hours = 23, minutes = 59, seconds = 59)
print(cutoff_w1)

train_w1 = train[train.ts <= cutoff_w1]
train_w1 = train_w1.reset_index(drop = True)
del train
gc.collect()

2022-08-07 21:59:59
CPU times: user 44.3 s, sys: 22.8 s, total: 1min 7s
Wall time: 1min 4s


62

In [4]:
#drop one_event_sessions
one_event_df = train_w1.groupby('session', as_index = False).count()
one_event_df = one_event_df[one_event_df['aid']==1]
one_sessions =  one_event_df['session']
train_w1 = train_w1[~train_w1['session'].isin(one_sessions)]
del one_event_df, one_sessions
gc.collect()

34

In [5]:
# #split and save parquets

# def split_file(train_w1=train_w1, splits = 5):
#     co = len(train_w1.session.unique())//splits
#     for i in range(splits):
#         if i != splits-1: temp = train_w1[(train_w1.session < co*(i+1)) & (train_w1.session >= co*i)]
#         else: temp = train_w1[train_w1.session >= co*i]
#         temp.to_parquet(f'{TEMP_PATH}df_w1_part{i}.parquet')
#         del temp
                                                                          
# split_file()
# del train_w1
# gc.collect() 

In [6]:
%%time

def get_train_target():
    
    train_w1_list = []
    label_w1 = []
    j=0
    end = len(train_w1.session.unique())
    

    for i, grp in tqdm(enumerate(train_w1.groupby('session'))):
        cutoff = np.random.randint(1, grp[1].shape[0]) 
        train_w1_list.append(grp[1].iloc[:cutoff])
        label_w1.append(grp[1].iloc[cutoff:])
        if (i % 500000 == 0) or (i == end - 1):
            train_w1_df = pd.concat(train_w1_list).reset_index(drop=True)
            label_w1 = pd.concat(label_w1).reset_index(drop=True)
            train_w1_df.to_parquet(f'/kaggle/working/train_w1_{j}.parquet')
            label_w1.to_parquet(f'/kaggle/working/label_w1_{j}.parquet')
            j += 1
            del train_w1_list, label_w1
            train_w1_list = []
            label_w1 = []          
            gc.collect()


get_train_target()

4441142it [38:47, 1908.21it/s]

CPU times: user 37min 42s, sys: 1min 7s, total: 38min 50s
Wall time: 38min 47s
